# Prediction based on CaDRReS-Sc pre-trained model
This notebook show an example of how load a pre-trained CaDRReS-SC model and predict drug response based on new data.

In [132]:
import sys, os, pickle
from collections import Counter
import importlib
from ipywidgets import widgets
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))

from cadrres_sc import pp, model, evaluation, utility
importlib.reload(model)
importlib.reload(evaluation)

<module 'cadrres_sc.evaluation' from 'c:\\Users\\carey\\Desktop\\CaDRReS-Sc\\cadrres_sc\\evaluation.py'>

# Read pre-trained model

In [49]:
model_dir = '../my_models/'

In [50]:
obj_function = widgets.Dropdown(options=['cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'], description='Objetice function')

In [51]:
#choose which model you have trained previously
display(obj_function)

Dropdown(description='Objetice function', options=('cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'),…

## Load the pre-trained model based on your selection


In [71]:
model_spec_name = obj_function.value
model_file = model_dir + '{}_param_dict.pickle'.format(model_spec_name)

cadrres_model = model.load_model(model_file)

226

# Read test data
Again, for this example we load GDSC dataset.
@TODO: GDSC dataset using only essential gene list?

Note: GDSC_exp.tsv can be downloaded from https://www.dropbox.com/s/3v576mspw5yewbm/GDSC_exp.tsv?dl=0

## Notes for other test data

You can apply the model to other gene expression dataset. The input gene expression matrix should have been normalized, i.e. **for each sample, expression values are comparable across genes**. 

In this example the gene expression matrix provided by GDSC is already normalized using RMA.

For RNA-seq data, read count should be normalized by gene length, using normalization methods such as TPM.

In [53]:
gene_exp_df = pd.read_csv('../data/GDSC/GDSC_exp.tsv', sep='\t', index_col=0)
gene_exp_df = gene_exp_df.groupby(gene_exp_df.index).mean()
print("Dataframe shape:", gene_exp_df.shape, "\n")
gene_exp_df.head(2)

Dataframe shape: (17419, 1018) 



,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE,,,,,,,,,,,,,,,,,,,,,
A1BG,6.208447,5.025810,5.506955,4.208349,3.399366,4.917872,3.828088,5.146903,3.107543,5.062066,...,4.272172,3.435025,4.930052,2.900213,4.523712,5.074951,2.957153,3.089628,4.047364,5.329524
A1CF,2.981775,2.947547,2.872071,3.075478,2.853231,3.221491,2.996355,2.893977,2.755668,2.985650,...,2.941659,3.155536,2.983619,3.118312,2.975409,2.905804,2.944488,2.780003,2.870819,2.926353


## Calculate fold-change
We normalized baseline gene expression values for each gene by computing fold-changes compared to the median value across cell-lines

In [54]:
cell_line_log2_mean_fc_exp_df, cell_line_mean_exp_df = pp.gexp.normalize_log2_mean_fc(gene_exp_df)

## Read essential genes list

Or in case you want your training using one specific set of genes.

In [55]:
ess_gene_list = utility.get_gene_list('../data/essential_genes.txt')

## Calculate kernel feature 

Based on all given cell line samples with gene expression profiles and a list of genes (e.g. essential gene list). This step might take a bit more time than usual.

In [19]:
test_kernel_df = pp.gexp.calculate_kernel_feature(cell_line_log2_mean_fc_exp_df, cell_line_log2_mean_fc_exp_df, ess_gene_list)

Calculating kernel features based on 591 common genes
(17419, 1018) (17419, 1018)
100 of 1018 (72.09)s
200 of 1018 (72.82)s
300 of 1018 (72.84)s
400 of 1018 (77.45)s
500 of 1018 (84.25)s
600 of 1018 (83.22)s
700 of 1018 (76.33)s
800 of 1018 (73.21)s
900 of 1018 (73.08)s
1000 of 1018 (73.12)s


In [56]:
print("Dataframe shape:", test_kernel_df.shape, "\n")
test_kernel_df.head(2)

Dataframe shape: (1018, 1018) 



,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
906826,1.000000,0.086192,0.041425,0.071927,0.202293,-0.029861,-0.120344,0.146482,0.312897,0.081833,...,0.311786,-0.193352,-0.117408,-0.025511,0.281567,0.250319,0.203752,0.034862,0.088211,0.064251
687983,0.086192,1.000000,0.162319,-0.085682,0.097823,0.102867,-0.121224,-0.069853,0.064681,0.065166,...,-0.022742,-0.020703,0.069880,-0.048270,-0.034882,0.063103,-0.032428,-0.100407,0.097560,0.080860


# Drug response prediction

In [57]:
print('Predicting drug response using CaDRReS: {}'.format(model_spec_name))
pred_df, P_test_df= model.predict_from_model(cadrres_model, test_kernel_df, model_spec_name)
print('done!')

Predicting drug response using CaDRReS: cadrres-wo-sample-bias
dict_keys(['W_P', 'W_Q', 'b_P', 'b_Q', 'mse_train_vals', 'drug_list', 'kernel_sample_list', 'sample_list_train'])
done!


Inspecting the model predictions and save the predictions

In [66]:
print(pred_df.shape)
pred_df.head()

(1018, 226)


Drug ID,1,1001,1003,1004,1005,1006,1007,1008,1009,1010,...,64,71,83,86,87,88,89,9,91,94
906826,5.563384,11.197551,-3.821002,-4.408046,5.178409,2.588082,-4.598678,3.602878,7.847899,5.524393,...,2.632777,5.726290,-1.746131,-0.964468,-1.843930,1.251239,4.443037,0.498741,6.651731,5.521437
687983,5.045389,10.939430,-6.302554,-5.901524,3.440855,0.884265,-6.762185,1.908613,6.819450,4.123607,...,2.186832,4.656423,-1.955774,-0.702130,-2.183857,0.758536,3.641444,0.029947,6.192258,5.570770
910927,-0.413329,10.462546,-7.883025,-7.209464,2.361014,-2.115585,-10.309696,2.329492,6.290312,0.749764,...,0.490583,1.953213,-2.916568,-1.417753,-3.718091,0.479225,1.267404,-1.395101,3.240233,1.712555
1240138,5.287357,12.024466,-4.435457,-3.625705,4.940827,1.635318,-4.192371,5.267210,8.282641,5.139955,...,3.425822,6.886184,0.510051,0.777601,0.455071,2.773001,4.848870,1.207380,6.788547,5.130849
1240139,4.352276,11.062858,-6.013392,-5.628539,3.747667,-0.025814,-6.809785,3.366330,7.368008,4.506888,...,2.448541,5.120632,-1.329515,-0.543658,-1.923090,1.577360,3.777439,0.193670,5.759233,4.669670


In [65]:
# P_test_df is the drugs and their latent vectors
print(P_test_df.shape)
P_test_df.head()

(1018, 10)


,1,2,3,4,5,6,7,8,9,10
906826,-0.804565,-0.914433,-0.181211,-0.473878,-0.124136,0.065443,-0.482438,0.826997,-0.572300,1.318430
687983,-0.332619,-0.304389,0.232101,0.426264,0.790567,-0.016804,-0.776426,0.778803,-1.276974,0.074862
910927,1.027017,-1.138005,1.851276,0.170602,-1.199878,1.805714,-1.479265,0.320495,0.125314,-2.607392
1240138,-0.314672,0.902809,0.961897,-0.764521,0.457254,2.021789,-0.239779,1.585581,0.688210,0.324732
1240139,-0.604717,-0.618175,1.043440,0.241055,-0.083026,1.715673,0.288715,0.675248,-0.240856,0.007152


In [60]:
print('Saving ' + model_dir + '{}_test_pred.csv'.format(model_spec_name))
pred_df.to_csv(model_dir + '{}_test_pred.csv'.format(model_spec_name))

Saving ../my_models/cadrres-wo-sample-bias_test_pred.csv


---

**Authors:** [Chayaporn Suphavilai](mailto:@.com), [Rafael Peres da Silva](), Genome Institute of Singapore, Nagarajan Lab, November, 2020

---

Reproducibility tips from https://github.com/jupyter-guide/ten-rules-jupyter

# Evaluation
Using Spearman correlation and NDCG to evaluate performance

In [142]:
cell_line_obs_df = pd.read_csv('../data/GDSC/gdsc_all_abs_ic50_bayesian_sigmoid_only9dosages.csv', index_col=0)
cell_line_sample_list = cell_line_obs_df.index.astype(str)
cell_line_sample_list = np.array([s for s in cell_line_sample_list if s in gene_exp_df.columns])
# convert indices to string for consistency in filtering
cell_line_sample_list = cell_line_sample_list.astype(str)
cell_line_obs_df.index = cell_line_obs_df.index.astype(str)

cell_line_obs_df = cell_line_obs_df.loc[cell_line_sample_list, cadrres_model['drug_list']]

pred_df = pred_df.loc[cell_line_sample_list, cadrres_model['drug_list']]

per_sample_df, per_drug_df = evaluation.calculate_spearman(cell_line_obs_df, pred_df, cell_line_sample_list, cadrres_model['drug_list'])
print(f"Average of sample spearman correlation: {np.nanmean(per_sample_df.values)}")
print(f"Average of drug spearman correlation: {np.nanmean(per_drug_df.values)}")

ndcg = evaluation.calculate_ndcg(cell_line_obs_df, pred_df)
print(f"Average of samples NDCG value: {np.nanmean(ndcg.values)}")

Average of sample spearman correlation: 0.3898356091895378
Average of drug spearman correlation: nan
Average of samples NDCG value: 0.6589016486801139
